In [1]:
%reset
import os
import datetime

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import ta

RawStockDataDirectory = 'StockData/Raw/'
ProcessedStockDataDirectory = 'StockData/Processed/'

In [2]:
allData = list()

for stockData in os.listdir(RawStockDataDirectory):
    allData.append(pd.read_csv(RawStockDataDirectory + stockData, usecols=[1,2,3,4,5]))

In [3]:
allData[0]

,Open,High,Low,Close,Volume
0,0.099874,0.100308,0.099874,0.099874,469033600
1,0.095098,0.095098,0.094663,0.094663,175884800
2,0.088149,0.088149,0.087715,0.087715,105728000
3,0.089886,0.090320,0.089886,0.089886,86441600
4,0.092492,0.092927,0.092492,0.092492,73449600
...,...,...,...,...,...
10616,138.119995,143.320007,137.899994,141.110001,81760300
10617,140.309998,143.160004,140.300003,142.529999,66435100
10618,140.889999,142.429993,138.809998,141.860001,65799300
10619,143.169998,144.250000,141.899994,143.960007,54105100


In [4]:
count = 1
for stock in allData:
    # Calculate moving averages
    stock['MA10'] = stock['Close'].rolling(window=10).mean()
    stock['MA20'] = stock['Close'].rolling(window=20).mean()
    stock['MA50'] = stock['Close'].rolling(window=50).mean()
    stock['MA100'] = stock['Close'].rolling(window=100).mean()
    stock['MA200'] = stock['Close'].rolling(window=200).mean()

    # Calculate RSI
    stock['RSI'] = ta.momentum.rsi(stock['Close'], window=14)

    # Calculate Bollinger Bands
    BB = ta.volatility.BollingerBands(stock['Close'])
    stock['BBup'], stock['BBmid'], stock['BBlow'] = BB.bollinger_hband(), BB.bollinger_mavg(), BB.bollinger_lband()

    # Calculate MACD
    MACD = ta.trend.MACD(stock['Close'])
    stock['MACD'], stock['MACDsig'], stock['MACDdiff'] = MACD.macd(), MACD.macd_signal(), MACD.macd_diff()

    # Calculate ROC
    stock['ROC'] = ta.momentum.ROCIndicator(stock['Close']).roc()

    # Calculate KAMA
    stock['KAMA'] = ta.momentum.kama(stock['Close'])

    # Calculate OBV
    stock['OBX'] = ta.volume.OnBalanceVolumeIndicator(close=stock['Close'], volume=stock['Volume']).on_balance_volume()

    # Calculate STOCH
    STOCH = ta.momentum.StochasticOscillator(close=stock['Close'], high=stock['High'], low=stock['Low'])
    stock['STOCH'] = STOCH.stoch()
    stock['STOCHsig'] = STOCH.stoch_signal()

    stock.dropna(inplace=True)
    stock.reset_index(drop=True, inplace=True)

    stock.to_csv(ProcessedStockDataDirectory + "Processed_" + str(count))
    count += 1

In [5]:
allData[0]

,Open,High,Low,Close,Volume,MA10,MA20,MA50,MA100,MA200,...,BBmid,BBlow,MACD,MACDsig,MACDdiff,ROC,KAMA,OBX,STOCH,STOCHsig
0,0.049937,0.050371,0.049937,0.049937,91728000,0.058448,0.064354,0.073681,0.086830,0.091919,...,0.064354,0.050105,-0.006029,-0.004929,-0.001100,-27.673372,0.053634,-87180800,2.173659,0.724553
1,0.052542,0.052976,0.052542,0.052542,94684800,0.057276,0.063485,0.073056,0.086391,0.091682,...,0.063485,0.048595,-0.006080,-0.005160,-0.000921,-22.930672,0.053436,7504000,15.216113,5.796591
2,0.052976,0.053411,0.052976,0.052976,49996800,0.056277,0.062421,0.072448,0.085948,0.091474,...,0.062421,0.047721,-0.006016,-0.005331,-0.000685,-19.736863,0.053367,57500800,18.180337,11.856703
3,0.052976,0.053411,0.052976,0.052976,61129600,0.055452,0.061292,0.071935,0.085527,0.091300,...,0.061292,0.047352,-0.005898,-0.005444,-0.000453,-17.567731,0.053317,118630400,20.511872,17.969441
4,0.057319,0.057753,0.057319,0.057319,45046400,0.055018,0.060576,0.071466,0.085149,0.091137,...,0.060576,0.047386,-0.005391,-0.005434,0.000042,-8.964986,0.053462,163676800,51.430994,30.041068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10417,138.119995,143.320007,137.899994,141.110001,81760300,134.794000,131.685499,139.215399,143.749921,148.521099,...,131.685499,123.288547,-0.039581,-1.694167,1.654586,11.672998,137.123860,224228370400,88.459500,86.734882
10418,140.309998,143.160004,140.300003,142.529999,66435100,136.032001,132.200499,139.368599,143.588753,148.378469,...,132.200499,122.561555,0.566284,-1.242077,1.808361,14.005761,139.050589,224294805500,95.743490,93.026100
10419,140.889999,142.429993,138.809998,141.860001,65799300,137.145001,132.700499,139.268399,143.437757,148.230945,...,132.700499,122.186354,0.981064,-0.797449,1.778513,9.442992,139.870324,224229006200,92.133587,92.112192
10420,143.169998,144.250000,141.899994,143.960007,54105100,138.192001,133.397000,139.153599,143.300373,148.104125,...,133.397000,121.884581,1.462376,-0.345484,1.807860,10.610844,141.036311,224283111300,98.502101,95.459726
